## Gift Contract Testing
### Sign message by Ethereum address

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from eth_account.messages import encode_defunct
import json
from subprocess import Popen, PIPE
import os

KEY_PHRASE = 'KEY PHRASE'
NUMBER_OF_PARTICIPANTS = 10
NICKNAME_LIST = ["john"]
AVATAR_CID_LIST = ["QmNqdEgYJwe8QSZvcpBXqcNoG9BRoSf71q6mJLe3aRhZAW"]
CLAIM_AMOUNT_LIST = [1477000000]
ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'

PASSPORT_CONTRACT_ADDRESS = "bostrom1nfmvw8x37w00p3geuu8lrt3vt5kadxa5xd9us7"
INITIAL_BALANCE = str(1_000_000_000)
COEF_UP = str(20)
COEF_DOWN = str(5)
COEF = str(0)
TARGET_CLAIM = str(100_000)


def execute_bash(bash_command: str):
    _output, _error = Popen(bash_command, shell=True, stdout=PIPE, stderr=PIPE).communicate()
    return _output.decode("utf-8"), _error.decode("utf-8")


claims_list = []
for _ in range(NUMBER_OF_PARTICIPANTS):
    claim_item = {'amount': CLAIM_AMOUNT_LIST[0]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], 'cosmos')
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    message_json = {
        "nickname": NICKNAME_LIST[0],
        "avatar_cid": AVATAR_CID_LIST[0],
        "gift_claiming_address_type": "ethereum",
        "gift_claiming_address": claim_item['ethereum_address'],
        "target_address": claim_item['bostrom_address'],
        "relay_reward": "0.01"
    }
    claim_item['message'] = json.dumps(message_json)

    # Sign message form Ethereum address
    signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    claim_item['message_signature'] = signed_message.signature.hex()

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['message_signature']).lower()

    claims_list.append(claim_item)

claims_df = pd.DataFrame(claims_list)
claims_df.to_csv('claims_ethereum_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

,amount,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,message_signature
0,1477000000,0x2bfe53b0b377cdf2c603f7d221feb9f3c1d7ecd1,0x45be96b69de4ea8cc7cfa6e02d5b36ec262cafd54895...,bostrom1gpcxup6j30akmp583h7yw6sdhj0uerxfu68w63,cosmos1gpcxup6j30akmp583h7yw6sdhj0uerxflfnayk,silly used brick stock couch lazy provide onio...,"{""nickname"": ""john"", ""avatar_cid"": ""QmNqdEgYJw...",0xd5c7d02abb82e4068bd8574201adb23b498209ee570d...
1,1477000000,0xe2a52413bd9e440d1b0b5eb81fea2c5106cc7521,0x4acc7775bd496b0ca2eebea731091242094dc7eef125...,bostrom167hjy7mq6v8u09aa4qxczmj8gea42eu8me37e7,cosmos167hjy7mq6v8u09aa4qxczmj8gea42eu8c29d8e,various cupboard useful unveil quote travel ra...,"{""nickname"": ""john"", ""avatar_cid"": ""QmNqdEgYJw...",0x2fb6e30e7fc1133062662d3af756d21833edea0f1b15...
2,1477000000,0x8d6fe30bb2409dc36166236c05b2d3818f9a9686,0xb92491e731124282d8fbed7c6951b7043538f9ce5776...,bostrom1h8ffjuc0pq5cfa5c8ws5tt4cug57ed80shgaze,cosmos1h8ffjuc0pq5cfa5c8ws5tt4cug57ed80nyuwu7,language lunch suspect stock inside episode vo...,"{""nickname"": ""john"", ""avatar_cid"": ""QmNqdEgYJw...",0x31e8fd8eb15a289c0e8ef3061e6551396d944834773c...
3,1477000000,0x922bac049290ef02eb2651bbd703cbc66af131d0,0x01a4bfccd78ca2b9c39dccdd9686c1e9f2c1d268cb16...,bostrom1qx3rcpzhfag7n63qcmpjdzw3kcgtkh2w3686cr,cosmos1qx3rcpzhfag7n63qcmpjdzw3kcgtkh2wjfnfxy,health play detail census come stairs ship tra...,"{""nickname"": ""john"", ""avatar_cid"": ""QmNqdEgYJw...",0xc74b9153dac3f8d69b2f5ee8a37b873900a375fa8045...
4,1477000000,0x77eb8667279364b8360595fcd5b8ad6006f978ad,0xc7bcdf751eac7520a7cb09f46ec7a1ac6a9c5055a1d8...,bostrom1l8uar2fk4n88y3yfxdq47ll2m92j8a57w8z674,cosmos1l8uar2fk4n88y3yfxdq47ll2m92j8a57d5kfqj,rich riot hollow fish loud print shine daughte...,"{""nickname"": ""john"", ""avatar_cid"": ""QmNqdEgYJw...",0xb07f21df92360f9c3580e4b8cdfced1b0c60b95a8e1e...
5,1477000000,0x296680f700465a64cd92708ab45d42b5312405ce,0x6fdc169a509ee721d8f9b788a94fa2acd8c0e9be58d5...,bostrom186hnt6az9jcjgpmwzc8avtqc0993auhzzjx28z,cosmos186hnt6az9jcjgpmwzc8avtqc0993auhzppjee9,actual settle survey lottery execute enable go...,"{""nickname"": ""john"", ""avatar_cid"": ""QmNqdEgYJw...",0x3fd0b3e6d2c1087421c32261b3e01fa0840966816498...
6,1477000000,0x2389cdb0db1a07f1b97678a27ce3a7b5ed0c7f1b,0x3f73d11fb1a35cfce6db583585ca1749dfd04ad58a38...,bostrom17zmth6ttym5fq6f78urxm3mqnhpsske5ylpag9,cosmos17zmth6ttym5fq6f78urxm3mqnhpsske58v4wkz,inch screen lion solve allow crucial mule arre...,"{""nickname"": ""john"", ""avatar_cid"": ""QmNqdEgYJw...",0x0bb9e31cd312b15b1e87917415266679e6f797148406...
7,1477000000,0x0d1d9cd875448cef18c7924e04dac4fd8807fa80,0x2993e82c28b3a873e28d60e4af79855bbd22d34ebf91...,bostrom1nfh0602jjwhwtq4848jafnng800h4prvyj3h9m,cosmos1nfh0602jjwhwtq4848jafnng800h4prv8p9ymu,coconut inch property minute property notice r...,"{""nickname"": ""john"", ""avatar_cid"": ""QmNqdEgYJw...",0x6336a4ed74bcc2aa6d8b99df143a0253e523609fb260...
8,1477000000,0xa58ebe0beaaed9efae699323c25ae2a1a800661a,0xcdc6f87c630aedceb7a5d977d4dbb91e2a38f7b499a2...,bostrom12jalux75962rg7n48vahev5z7u8jnh4e05fh24,cosmos12jalux75962rg7n48vahev5z7u8jnh4ev8ay5j,fish duty van alley sheriff purpose collect se...,"{""nickname"": ""john"", ""avatar_cid"": ""QmNqdEgYJw...",0x886f8f3d248124ace3d648fc9a386e5609b887870275...
9,1477000000,0x169eac8150e050bf548bef2a658cc936dbcd4b00,0x826f371ade2880433b76fdc2d3b772840610d4838806...,bostrom18qr2aycn463xhxhlzhn65tsrus8hdklfd4z2v5,cosmos18qr2aycn463xhxhlzhn65tsrus8hdklfwxkejn,rug wrap random help one renew warfare goddess...,"{""nickname"": ""john"", ""avatar_cid"": ""QmNqdEgYJw...",0xbcd228bfbbf0908f20d13fff5a6212807f103e43d84e...


### Instantiate Gift Contract

In [2]:
init_output, init_error = execute_bash(
    f'''INIT='{{"owner":"{os.getenv('WALLET_ADDRESS')}", "passport":"{PASSPORT_CONTRACT_ADDRESS}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF}", "target_claim":"{TARGET_CLAIM}"}}' \
           && cyber tx wasm instantiate 9 "$INIT" --from $WALLET --amount 1000000000boot --label "cybergift test" -y --chain-id=space-pussy-1 --gas 3500000 --broadcast-mode block -o json --node=https://rpc.space-pussy-1.cybernode.ai:443''')
if init_error:
    print(init_error)
init_json = json.loads(init_output)
contract_address = \
    [event['attributes'][0]['value'] for event in init_json['logs'][0]['events'] if event['type'] == 'instantiate'][0]
print(f'Contract address: {contract_address}')

Contract address: bostrom1gu70g4jnj93k6uq77mc450mvz5780zqy22526e2l4yhehcgawy0qe2dqnq


### Create Merkle Root

In [4]:
root_source_list = [{'address': item['ethereum_address'],
                     'amount': str(claim_item['amount'])} for item in claims_list]

with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [5]:
execute_bash(f'rm {ROOT_FILE_NAME}')
root_output, root_error = execute_bash(
    f'merkle-airdrop-cli generateRoot --file {ROOT_SOURCE_FILE_NAME} >> {ROOT_FILE_NAME} && cat {ROOT_FILE_NAME}')
root = root_output.replace('\n', '')
print(f'Merkle root: {root}')

Merkle root: 0bd17d307080b54a0034fc0e33d246556ae6d15ccf5d97d6238824f23e55c4a2


In [6]:
root_register_output, root_register_error = \
    execute_bash(
        f'''REGISTER='{{"register_merkle_root":{{"merkle_root":"{root}"}}}}' \
            && CONTRACT="{contract_address}" \
            && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET --broadcast-mode block -y --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443''')

#### Get Merkle Root form the Gift Contract

In [7]:
print(
    execute_bash(
        f'''QUERY='{{"merkle_root": {{}}}}' \
            && cyber query wasm contract-state smart {contract_address} "$QUERY" --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443''')[
        0])

data:
  merkle_root: 0bd17d307080b54a0034fc0e33d246556ae6d15ccf5d97d6238824f23e55c4a2



### Generate Proofs

In [21]:
proof_output, proof_error = execute_bash(f'merkle-airdrop-cli generateProofs --file {ROOT_SOURCE_FILE_NAME} --address {claims_df.ethereum_address[0]} --amount {claims_df.amount[0]}')
proof = proof_output.strip('][').replace('\n','').replace(" ",'').replace("'",'').replace("]",'').split(',')
proof

['c14b9d3c4d5f343f0a383a8b22afd273ce235576c0eb55c0d887ebc117a6a43e',
 'a69216a64da01cb8bb89585646f86bb306a2333a1852cf9c166d5873c644edfa',
 '3f7f3d34e3c87a42a1748a74ca6442e20c2d245d6593144075a65f7378e135b2',
 'a821f581bad8a673b1d1f72f4577d8bf4ddecebf0db948231d1a75f0af203bc6']

### Claim

In [22]:
print(
    execute_bash(
        f'''CLAIM='{{"claim":{{"claim_msg":{{"nickname":"{json.loads(claims_df.message[0])['nickname']}", "avatar_cid": "{json.loads(claims_df.message[0])['avatar_cid']}", "gift_claiming_address_type":"ethereum", "gift_claiming_address":"{claims_df.ethereum_address[0]}", "target_address":"{claims_df.bostrom_address[0]}", "relay_reward":"{json.loads(claims_df.message[0])['relay_reward']}"}}, "signature":"{claims_df.message_signature[0]}", "claim_amount":"{claims_df.amount[0]}", "proof":"{proof}"}}}}' \
            && cyber tx wasm execute {contract_address} "$CLAIM" --from $WALLET --broadcast-mode block -y --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443''')[0]
)

code: 5
codespace: wasm
data: ""
gas_used: "124255"
gas_wanted: "200000"
height: "800540"
info: ""
logs: []
raw_log: 'failed to execute message; message index: 0: Error parsing into type cw_cyber_airdrop::msg::ExecuteMsg:
  Invalid type: execute wasm contract failed'
timestamp: ""
tx: null
txhash: 32E4F626BDEF0ADD5683D70AD38197370D4471617F11158C37900F5AD3F705A0



### Sign message by Cosmos address
*TODO: add script for signature generation. You can sign a message only in the Keplr extension now*


message:
```json
{"nickname":"bob-cosmos","avatar_cid":"QmU1Nf2opJGZGNWmqxAa9bb8X6wVSHRBDCY6nbm3RmVXGb","gift_claiming_address_type":"cosmos","gift_claiming_address":"cosmos1qjjjur5et06eszh82pfv7kvdnu3sur23x00t3g","target_address":"bostrom1qjjjur5et06eszh82pfv7kvdnu3sur239umc00","relay_reward":"0.01"}
```
signature:
`CvMkqkQHVPV3DTyVErth16OdTjAwqQD6t+r9ImSpdwUZFin+UPeGSfH9hhuAmqYAp4CffhNNSEisdfzwwvlN/w==`
